# Chrysalis Unpacking Walkthrough (Notebook)

This notebook demonstrates the **offline + emulation-assisted** steps we used to unpack the Lotus Blossom/Chrysalis sample chain described by Rapid7.

It focuses on three practical artifacts:
- `input/encrypted_shellcode.bin` (Rapid7 `BluetoothService`): extract + decrypt RC4 config (offline)
- `input/log.dll`: emulate `LogWrite` to decrypt the stage1 buffer (Unicorn-based, still 100% Python)
- Decrypt and materialize the **main module** into a PE-like memory image / patched PE (offline transform)

## Requirements
- Install dependencies into the same Python environment your Jupyter kernel uses: `unicorn`, `pefile`, `capstone`, `bokeh` (see `requirements.txt`).
- Inputs must exist under `input/`:
  - `input/log.dll`
  - `input/encrypted_shellcode.bin`
  - `input/BluetoothService.exe`

Outputs are written under `output/` (created if missing).

## Optional plots
This notebook tries to use:
- `bokeh` if available
- else it will fall back to text summaries.

If you want interactive plots, install `bokeh` into the same Python environment your Jupyter kernel is using.


In [ ]:
from __future__ import annotations

import hashlib
import struct
import sys
from pathlib import Path

# Ensure repo root is on sys.path even if Jupyter started in notebooks/.
_cwd = Path.cwd().resolve()
for _d in [_cwd, *_cwd.parents]:
    if (_d / 'scripts' / 'chrysalis_notebook_lib.py').exists():
        sys.path.insert(0, str(_d / 'scripts'))
        break

from chrysalis_notebook_lib import (
    ArgStruct,
    ConfigExtractor,
    MainModuleMaterializer,
    Stage1Decryptor,
    find_repo_root,
    sha256_bytes,
    shannon_entropy,
)

ROOT = find_repo_root(Path.cwd())
INPUT_DIR = ROOT / 'input'
OUTPUT_DIR = ROOT / 'output'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

LOG_DLL = INPUT_DIR / 'log.dll'
ENC_SHELLCODE = INPUT_DIR / 'encrypted_shellcode.bin'
CONTAINER_EXE = INPUT_DIR / 'BluetoothService.exe'

for _p in [LOG_DLL, ENC_SHELLCODE, CONTAINER_EXE]:
    assert _p.exists(), f'Missing input file: {_p}'

def sha256_path(p: Path) -> str:
    return hashlib.sha256(p.read_bytes()).hexdigest()

print('ROOT:', ROOT)
print('log.dll sha256:', sha256_path(LOG_DLL))
print('encrypted_shellcode.bin sha256:', sha256_path(ENC_SHELLCODE))
print('BluetoothService.exe sha256:', sha256_path(CONTAINER_EXE))


In [ ]:
def sha256_bytes(b: bytes) -> str:
    return hashlib.sha256(b).hexdigest()

def sha256_path(p: Path) -> str:
    return sha256_bytes(p.read_bytes())

def shannon_entropy(b: bytes) -> float:
    if not b:
        return 0.0
    counts = [0] * 256
    for x in b:
        counts[x] += 1
    import math
    n = len(b)
    ent = 0.0
    for c in counts:
        if c == 0:
            continue
        p = c / n
        ent -= p * math.log2(p)
    return ent

def hexdump(b: bytes, start: int = 0, length: int = 0x100) -> str:
    chunk = b[start:start+length]
    out = []
    for i in range(0, len(chunk), 16):
        row = chunk[i:i+16]
        hs = " ".join(f"{x:02x}" for x in row)
        asc = "".join(chr(x) if 32 <= x < 127 else "." for x in row)
        out.append(f"{start+i:08x}  {hs:<47}  {asc}")
    return "\n".join(out)

print("log.dll sha256:", sha256_path(LOG_DLL))
print("encrypted_shellcode.bin sha256:", sha256_path(ENC_SHELLCODE))
print("BluetoothService.exe sha256:", sha256_path(CONTAINER_EXE))

## 1) Config Extraction + RC4 Decryption (Offline)

Rapid7 notes the encrypted configuration is stored in `BluetoothService` (our `input/encrypted_shellcode.bin`) at:
- offset `0x30808`
- size `0x980`
- RC4 key `qwhvb^435h&*7`

We'll implement RC4 in Python and show recovered plaintext strings.

In [ ]:
def rc4_crypt(data: bytes, key: bytes) -> bytes:
    # Classic RC4 KSA+PRGA
    S = list(range(256))
    j = 0
    for i in range(256):
        j = (j + S[i] + key[i % len(key)]) & 0xFF
        S[i], S[j] = S[j], S[i]
    i = 0
    j = 0
    out = bytearray(len(data))
    for n, x in enumerate(data):
        i = (i + 1) & 0xFF
        j = (j + S[i]) & 0xFF
        S[i], S[j] = S[j], S[i]
        K = S[(S[i] + S[j]) & 0xFF]
        out[n] = x ^ K
    return bytes(out)

cfg_off = 0x30808
cfg_len = 0x980
cfg_key = b"qwhvb^435h&*7"

enc = ENC_SHELLCODE.read_bytes()
assert cfg_off + cfg_len <= len(enc)
cfg_enc = enc[cfg_off:cfg_off+cfg_len]
cfg_plain = rc4_crypt(cfg_enc, cfg_key)

cfg_out = OUTPUT_DIR / "notebook_config_decrypted.bin"
cfg_out.write_bytes(cfg_plain)

print("Encrypted cfg sha256:", sha256_bytes(cfg_enc), "entropy", f"{shannon_entropy(cfg_enc):.3f}")
print("Decrypted cfg sha256:", sha256_bytes(cfg_plain), "entropy", f"{shannon_entropy(cfg_plain):.3f}")
#print("Wrote:", cfg_out)
print()
print("First 0x120 bytes of decrypted config:")
print(hexdump(cfg_plain, 0, 0x120))

In [ ]:
def extract_printable_ascii(b: bytes, min_len: int = 6) -> list[str]:
    out = []
    cur = bytearray()
    for x in b:
        if 32 <= x < 127:
            cur.append(x)
        else:
            if len(cur) >= min_len:
                out.append(cur.decode("ascii", errors="ignore"))
            cur.clear()
    if len(cur) >= min_len:
        out.append(cur.decode("ascii", errors="ignore"))
    return out

print("ASCII strings in decrypted config (min_len=6):")
for s in extract_printable_ascii(cfg_plain, 6):
    print("-", s)

## 2) Stage1 Decryption via log.dll (Emulation-Assisted)

The stage1 buffer is decrypted by `log.dll` at runtime.

Instead of re-implementing the full LCG-based decrypt routine from scratch, we run the **actual malware decryption function** inside a controlled x86 emulator (Unicorn) and dump the decrypted bytes.

This is still *pure Python* end-to-end, and it's reproducible on macOS/ARM.

In [ ]:
# Run the Unicorn-based log.dll emulator directly (in-kernel).
stage1_runner = Stage1Decryptor()
stage1_runner.run(
    log_dll_path=LOG_DLL,
    encrypted_payload_path=ENC_SHELLCODE,
    output_dir=OUTPUT_DIR,
    mode='logwrite',
    stop_at='none',
)

stage1 = OUTPUT_DIR / 'shellcode.bin'
stage1_full = OUTPUT_DIR / 'shellcode_full.bin'
assert stage1.exists(), f'Missing {stage1}'
assert stage1_full.exists(), f'Missing {stage1_full}'

b1 = stage1.read_bytes()
bfull = stage1_full.read_bytes()
print('stage1 len:', len(b1), 'sha256:', sha256_bytes(b1), 'entropy:', f'{shannon_entropy(b1):.3f}')
print('stage1_full len:', len(bfull), 'sha256:', sha256_bytes(bfull), 'entropy:', f'{shannon_entropy(bfull):.3f}')


## 3) Main Module Materialization (Offline Transform)

Rapid7 describes the main module decryption as a simple byte transform using key `gQ2JR&9;`.

The repo script `offline_extract_stage2.py` does two useful things:
- injects the stage1-provided regions into a known PE container (`BluetoothService.exe`) at 5 RVAs
- applies the decrypt transform to those regions

We generate two artifacts:
- `output/main_module_patched.exe` (easy to open as PE, signature invalid after modification)
- `output/main_module_mem.bin` (decrypted memory image; often the best artifact for RE)


### 3a) The Byte Transform (Rapid7 key `gQ2JR&9;`)

Rapid7's pseudocode per byte is:

```
x = x + k
x = x ^ k
x = x - k
```

A useful property (for this sample) is that this transform is its own inverse.
So applying it twice yields the original byte stream again.

(In our scripts you may see “rounds=5”; for this transform, odd rounds behave like 1 round, even rounds undo it.)


In [ ]:
KEY = b"gQ2JR&9;"

def bxform_byte(x: int, k: int) -> int:
    x = (x + k) & 0xFF
    x = x ^ k
    x = (x - k) & 0xFF
    return x

def bxform(data: bytes, key: bytes, rounds: int = 1) -> bytes:
    out = bytearray(data)
    for _ in range(rounds):
        for i in range(len(out)):
            k = key[i & 7]
            out[i] = bxform_byte(out[i], k)
    return bytes(out)

# Quick sanity: involution property on a short prefix
test = b"hello world" * 32
t1 = bxform(test, KEY, rounds=1)
t2 = bxform(t1, KEY, rounds=1)
assert t2 == test

print("byte-transform sanity: ok (transform is an involution)")
print("rounds=1 == rounds=5 ?", bxform(test, KEY, rounds=1) == bxform(test, KEY, rounds=5))
print("rounds=1 == rounds=3 ?", bxform(test, KEY, rounds=1) == bxform(test, KEY, rounds=3))
print("rounds=2 restores original ?", bxform(test, KEY, rounds=2) == test)

In [ ]:
# This arg-struct is sample-specific and comes from stage1 runtime data.
# For the Rapid7 sample it is stable and matches the described layout (5 regions).
ARG_STRUCT = [
    0x116A7, 0x5, 0x1000, 0x24000, 0x2D000,
    0x30000, 0x31000, 0x0, 0x0, 0x23000,
    0x8E00, 0xC00, 0x200, 0x1C00, 0x0,
    0x0, 0x400000, 0x0, 0x31000, 0x2C5D0,
    0x30001000, 0x100014C0, 0x100014D0, 0x30000000, 0x0,
]
arg_struct = ArgStruct.from_iterable(ARG_STRUCT)

patched_exe = OUTPUT_DIR / 'main_module_patched.exe'
mem_img = OUTPUT_DIR / 'main_module_mem.bin'

mat = MainModuleMaterializer(key=b'gQ2JR&9;', rounds=1)
info = mat.materialize(
    container_pe_path=CONTAINER_EXE,
    stage1_full_path=stage1_full,
    arg_struct=arg_struct,
    out_patched_pe_path=patched_exe,
    out_mem_image_path=mem_img,
)

print('patched_exe:', patched_exe, 'sha256:', info['patched_pe_sha256'], 'size:', patched_exe.stat().st_size)
print('mem_img:   ', mem_img, 'sha256:', info['mem_image_sha256'], 'size:', mem_img.stat().st_size)
print('regions:', [(hex(r), hex(s)) for (r,s) in info['regions']])


In [ ]:
def parse_pe_minimal(path: Path):
    b = path.read_bytes()
    if b[:2] != b"MZ":
        raise ValueError("Not MZ")
    pe = struct.unpack_from("<I", b, 0x3C)[0]
    if b[pe:pe+4] != b"PE\0\0":
        raise ValueError("Not PE")
    coff = pe + 4
    opt = coff + 20
    magic = struct.unpack_from("<H", b, opt)[0]
    if magic != 0x10B:
        raise ValueError(f"Unexpected PE magic {magic:#x}")
    ep_rva = struct.unpack_from("<I", b, opt + 0x10)[0]
    imgbase = struct.unpack_from("<I", b, opt + 0x1C)[0]
    szimg = struct.unpack_from("<I", b, opt + 0x38)[0]
    return {
        "image_base": imgbase,
        "entry_rva": ep_rva,
        "entry_va": imgbase + ep_rva,
        "size_of_image": szimg,
    }

print("patched exe PE summary:")
print(parse_pe_minimal(patched_exe))

print("\nmem image PE summary:")
print(parse_pe_minimal(mem_img))

## 4) Quick Visualizations

We'll compare entropy and a byte histogram for:
- encrypted input (`encrypted_shellcode.bin`)
- decrypted stage1 (`shellcode.bin`)
- decrypted main module memory image (`main_module_mem.bin`)

If `bokeh` is unavailable, we'll print text summaries instead.

In [ ]:
samples = {
    "encrypted_shellcode.bin": ENC_SHELLCODE.read_bytes(),
    "stage1_shellcode.bin": stage1.read_bytes(),
    "main_module_mem.bin": mem_img.read_bytes(),
}

for name, b in samples.items():
    print(f"{name:24s} len={len(b):8d} sha256={sha256_bytes(b)} ent={shannon_entropy(b):.3f}")

try:
    from bokeh.io import output_notebook
    from bokeh.plotting import figure, show

    output_notebook()

    def hist_counts(b: bytes):
        c = [0] * 256
        for x in b:
            c[x] += 1
        return c

    p = figure(
        title="Byte Histogram (0..255)",
        width=1100,
        height=320,
        x_axis_label="byte value",
        y_axis_label="count",
        tools="pan,wheel_zoom,box_zoom,reset,save,hover",
    )
    p.toolbar.logo = None

    x = list(range(256))
    palette = [
        "#1f77b4",
        "#ff7f0e",
        "#2ca02c",
        "#d62728",
        "#9467bd",
        "#8c564b",
    ]
    for i, (name, b) in enumerate(samples.items()):
        y = hist_counts(b)
        p.line(x, y, line_width=2, alpha=0.85, legend_label=name, color=palette[i % len(palette)])

    p.legend.location = "top_right"
    p.legend.click_policy = "hide"
    show(p)

except Exception as e:
    print("bokeh not available:", e)
    print("(skipping plots)")

## Next Steps (Manual RE)

If you're opening the decrypted main module in IDA:
- prefer `output/main_module_mem.bin` (memory image)
- or open `output/main_module_patched.exe` and jump to the EntryPoint.

You can also re-run the emulator in `--mode full` for additional runtime telemetry, but the analysis here intentionally avoids a full stage1 emulation because the shellcode uses exception-driven / anti-analysis control flow.
